<a href="https://colab.research.google.com/github/negodinaarina/sem_homework/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_1_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 файлам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** c - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


# Задание 1 (3 балла)

1. Возьмите 10 000 случайных строк из таблицы sales_train и найдите в них долю товаров из категории 37. 
2. Найдите самый удачный по общему количеству продаж день для магазина St. Petersburg Nevsky Center shopping center за 2014 год.
3. Есть ли товар, который продается успешнее всех остальных (по ежемесячному количеству) во всех магазинах?

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))
items = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/items.csv'))
merged = pd.merge(sales, items).sample(10000)
print(f"Доля товаров с категорией 37 в 10 000 случайных строк - {len(merged[merged['category_id']==37])/10000}")


Доля товаров с категорией 37 в 10 000 случайных строк - 0.0619


In [ ]:
shops = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/shops.csv'))
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))
id = shops[shops.shop_name == ' St. Petersburg Nevsky Center shopping center']['shop_id']
sales['date'] = pd.to_datetime(sales['date'], dayfirst=True)
sales14 = (sales["date"] > '01-01-2014') & (sales["date"] <= '31-12-2014') & (sales["shop_id"] == int(id))
sales14 = sales.loc[sales14]
sales14 = sales14[['date', 'item_cnt_day']].groupby(['date']).sum()
print(sales14.nlargest(1, 'item_cnt_day'))

            item_cnt_day
date                    
2014-12-27         588.0


/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:52: UserWarning: Parsing '31-12-2014' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._cmp_method(other, operator.le)


In [ ]:
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))
items = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/items.csv'))
sales_g = sales[['date_block_num', 'item_id', 'item_cnt_day']].groupby(["date_block_num", "item_id"]).sum().reset_index()
sales_g = sales_g.sort_values(by='item_cnt_day', ascending=False)
sales_g.drop_duplicates(subset=['date_block_num'])
l = list(sales_g['item_id'])
f = True
for i in range(1, len(l)):
  if l[i-1] != l[i]:
    f = False
    break
if f:
  print(f'Есть товар, который продавался успешнее всех остальных каждый месяц - {l[0]}')
else:
  print('Такого товара нет')
# print(sales_g.drop_duplicates(subset=['date_block_num']))


# Задание 2 (2 балла)

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))[['item_id', 'item_cnt_day']].groupby(['item_id']).sum().reset_index().sort_values(by='item_cnt_day', ascending=False).head(10)
items = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/items.csv'))
print('Самые продающиеся товары:')
for item in sales.item_id:
  print(items.loc[items['item_id'] == item].values[0])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Самые продающиеся товары:
['Corporate package T-shirt 1C Interest white (34 * 42) 45 microns' 20949
 71]
['Diablo III [PC, Jewel, Russian version]' 2808 30]
['Grand Theft Auto V [PS3, Russian subtitles]' 3732 19]
['Acceptance of funds for 1C-Online' 17717 79]
['Playstation Store replenishment of wallet: Payment card 1000 rubles.'
 5822 35]
['Grand Theft Auto V [Xbox 360, Russian subtitles]' 3734 23]
['Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1108A/B01)' 6675 12]
['Grand Theft Auto V [PC, Russian subtitles]' 3731 30]
['Battlefield 4 [PC, Russian version]' 1855 30]
['Some of us [PS3, Russian version]' 16787 19]


In [ ]:
ids = sales['item_id']
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))[['date', 'item_id']]
sales['date'] = pd.to_datetime(sales['date'], dayfirst=True)
merged = pd.merge(ids, sales)
merged = merged.groupby(merged.date.dt.year)['item_id'].apply(set).reset_index(name="item_id")


In [ ]:
set2013 = merged.at[0, 'item_id']
set2014 = merged.at[1, 'item_id']
set2015 = merged.at[2, 'item_id']
ids = set2013.intersection(set2014, set2015)
for id in ids:
  print(items.at[id, 'item_name'])
# print(ids)

{6675, 3732, 17717, 20949, 3734, 2808, 16787, 5822, 1855}


# Задание 3 (3 балла)

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

*Если таких категорий несколько, выведите все.*

In [43]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
cats = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/item_categories.csv'))
items = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/items.csv'))
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))[['item_id', 'item_cnt_day']].groupby(["item_id"]).sum().reset_index()
sales['category'] = 0
for item in sales.item_id:
  cat_id = items[items.item_id == item]['category_id'].values[0]
  sales.loc[sales['item_id'] == item, ['category']] = cat_id
sales = sales[['category', 'item_cnt_day']].groupby(['category']).sum().reset_index()
sales = sales.sort_values(by='item_cnt_day', ascending=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
sales = sales.reset_index(drop=True)
minc = sales.loc[len(sales) - 1, 'item_cnt_day']
maxc = sales.loc[0, 'item_cnt_day']
max_c = []
min_c = []
max_c.append(sales.at[0, 'category'])
min_c.append(sales.at[len(sales) - 1, 'category'])
for i in range(1, len(sales) - 1):
  if sales.loc[i, 'item_cnt_day'] == maxc:
    max_c.append(sales.loc[i, 'category'])

for i in range(len(sales) - 2, 0, -1):
  if sales.loc[i, 'item_cnt_day'] == minc:
    min_c.append(sales.loc[i, 'category'])

print('Самые продаваемые категории:')
for id in max_c:
  name = cats.loc[cats['item_category_id'] == id]['item_category_name'].values[0]
  print(f'Категория товара - {name}, id - {id}')

print('Самые непродаваемые категории:')
for id in min_c:
  name = cats.loc[cats['item_category_id'] == id]['item_category_name'].values[0]
  print(f'Категория товара - {name}, id - {id}')


Самые продаваемые категории:
Категория товара -  Cinema - DVD, id - 40
Самые непродаваемые категории:
Категория товара -  Books - Cognitive literature, id - 51
Категория товара -  Game consoles - PS2, id - 10


# Задание 4 (3 балла)

1. Для категорий из задания 3 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами. Оставьте в таблице только те товары, среднее количество продаж которых строго больше 100.
2. Найдите месяц с максимальными продажами для каждого товара из пункта 1.


In [46]:
cats = max_c + min_c
df = pd.DataFrame(cats, columns=['category_id'])
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))
items = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/items.csv'))
df = pd.merge(sales, items)[['date', 'date_block_num', 'shop_id', 'item_id', 'item_cnt_day', 'category_id']]
iwannasurvive = df.loc[df['category_id']. isin(cats)].groupby(['date', 'shop_id', 'item_id', 'category_id', 'date_block_num']).sum().reset_index()
top_shops = iwannasurvive[['shop_id', 'item_cnt_day']].groupby(['shop_id']).sum().reset_index().sort_values(by='item_cnt_day', ascending=False).head(5)
top5 = iwannasurvive.loc[iwannasurvive['shop_id']. isin(top_shops['shop_id'])][['item_id', 'item_cnt_day', 'date']].groupby(['item_id']).sum().sort_values(by='item_cnt_day', ascending=False)
print(top5)


         item_cnt_day
item_id              
21404          1486.0
10298          1363.0
21440          1359.0
11496          1122.0
13811          1064.0
...               ...
8143              1.0
14395             1.0
14399             1.0
14414             1.0
0                 1.0

[4716 rows x 1 columns]


# Задание 5 (3 балла)

1. Для категорий из задания 3 найдите среднюю и медианную цену по категории в пределах каждого месяца. Выведите топ-10 товаров по медиане из каждой категории.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [ ]:
# PUT YOUR CODE HERE